#### Tune BERT Model
https://huggingface.co/docs/transformers/training

In [3]:
from datasets import load_dataset
from datasets import DatasetDict

In [4]:
dataset = load_dataset("yelp_review_full")

In [5]:
dataset 

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

#### Sample Dataset
- only use 5% for demonstration purposes

In [6]:
sampled_dataset = DatasetDict({"train":dataset["train"].shuffle(seed=42).select(range(3000)), 
                               "test":dataset["test"].shuffle(seed=42).select(range(1000))})

In [7]:
sampled_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

### Tokenize Data

In [8]:
from transformers import BertTokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [11]:
tokenized_datasets = sampled_dataset.map(tokenize_function, batched=True)

### Train Model

In [12]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

import numpy as np
import evaluate


In [13]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
training_args = TrainingArguments(output_dir="test_trainer")
metric = evaluate.load("accuracy")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()